In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data=pd.read_csv("netflix_titles.csv")
data.shape
data.info()
data.corr()
data.iloc[:,1:5].head(1)## datada girdi kolonları

In [ ]:
###################################  Şablon  ##################################

## LabelEncoder kategoric veriyi sayısala dönüştürüyoruz
from sklearn import preprocessing
data=pd.read_csv("netflix_titles.csv")
lb=preprocessing.LabelEncoder()

yedekKolon1=lb.fit_transform(data[["show_id"]])
data["show_id"]=yedekKolon1

##OneHotEncoder Şablon kod
data_yedek=pd.read_csv("netflix_titles.csv")
yedek=data_yedek.iloc[:,1:2].values##1:2 -> yani type kolonu
lb=preprocessing.LabelEncoder()
yedek[:,0]=lb.fit_transform(data_yedek.iloc[:,1]) ##data_yedek.iloc[:,1] -> type kolonu 

ohe=preprocessing.OneHotEncoder()
yedek=ohe.fit_transform(yedek).toarray()
print(yedek)

yedek=pd.DataFrame(yedek)
yedek.head(20)

##Backward Elimination
import statsmodels.api as sm

data["ekKolon B0 için"]=np.ones(5189)
data_BW=data.iloc[:,[0,1,2,3,4,6]].values
data_BW=np.array(data_BW,dtype=float)
model=sm.OLS(data.iloc[:,5],data_BW).fit()
print(model.summary())
## 0.05(hata payı P>|t|) den büyük olan kolonları çıkarıp tekrar kontrol ediyoruz.


##Linear-MultiLinear(more column than one) Predict
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

x_train, x_test, y_train, y_test = train_test_split(data.iloc[:,0:5] , data["rating"] , test_size=0.33,random_state=0)
LR=LinearRegression()
LR.fit(x_train,y_train)
y_pred=LR.predict(x_test)


##sort and plot
df=pd.DataFrame(data=y_test)
df["predict"]=y_pred
df=df.sort_values(by=["rating"])

fig, ax = plt.subplots()
ax.scatter(range(0,1713,1), df2["rating"],color="red",label="Gerçek Değerler")
ax.plot(range(0,1713,1), df2["predict"],color="blue",label="Predict Eğrisi")
ax.set(xlabel='Test Verileri', ylabel='Rating',title='Multi Linear Regresyon')
ax.grid()
plt.legend()
plt.show()


## Polynomial Regression###########################################Bu Yanlış
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
polyReg=PolynomialFeatures(degree=2)#####
X=x_test.values
Y=y_test.values
Y=Y.reshape(-1,1)
x_poly=polyReg.fit_transform(X)

linReg=LinearRegression()
linReg.fit(x_poly,y_test)

pred=linReg.predict(polyReg.fit_transform(X))
pred=pd.DataFrame(pred)
pred=pred.sort_values(by=0)
##Polynomial Regression###########################################Bu DOĞRU
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
polyReg=PolynomialFeatures(degree=5)
X=x_train.values
Y=y_train.values
x_poly=polyReg.fit_transform(X)

linReg=LinearRegression()
linReg.fit(x_poly,y_train)

predPoly=linReg.predict(polyReg.fit_transform(x_test.values))



## Scaler Ölçekleme
from sklearn.preprocessing import StandardScaler
X=x_test.values
Y=y_test.values

sc1=StandardScaler()
x_olcekli=sc1.fit_transform(X)
sc2=StandardScaler()
y_olcekli=sc2.fit_transform(Y.reshape(-1,1))

##Support Vector Regressiom *Ölçekli veri kullanmamız gerekiyor
from sklearn.svm import SVR
svr_reg=SVR(kernel='rbf')
svr_reg.fit(x_olcekli,y_olcekli)

df4=pd.DataFrame(data=y_olcekli)
df4["predict"]=svr_reg.predict(x_olcekli)
df4=df4.rename(columns={0: "rating"})
df4=df4.sort_values(by="rating")

fig, ax4 = plt.subplots()
ax4.scatter(range(0,1713,1), df4["rating"],color="red",label="Gerçek Değerler")
ax4.plot(range(0,1713,1), df4["predict"],"b",label="Predict Eğrisi")
ax4.set(xlabel='Test Verileri', ylabel='Rating',
       title='SVR-rbf')
plt.legend()
plt.show()


##DecisionTree Regressor
from sklearn.tree import DecisionTreeRegressor
DT=DecisionTreeRegressor(random_state=0)
X=x_test.values
Y=y_test.values
DT.fit(X,Y)

dfDT=pd.DataFrame(data=y_test)
dfDT["predict"]=DT.predict(X)
dfDT=dfDT.rename(columns={0: "rating"})
dfDT=dfDT.sort_values(by=["rating"])

fig, ax = plt.subplots()
ax.scatter(range(0,1713,1), dfDT["rating"],color="red",label="Gerçek Değerler")
ax.plot(range(0,1713,1), dfDT["predict"],color="blue",label="Predict Eğrisi")
ax.set(xlabel='Test Verileri', ylabel='Rating',title='Multi Linear Regresyon')
ax.grid()
plt.legend()
plt.show()

##Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor
RF=RandomForestRegressor(n_estimators=20,random_state=0)
X=x_test.values
Y=y_test.values
RF.fit(X,Y.ravel())

dfRF=pd.DataFrame(data=y_test)
dfRF["predict"]=RF.predict(X)
dfRF=dfRF.rename(columns={0: "rating"})
dfRF=dfRF.sort_values(by=["rating"])

fig, ax = plt.subplots()
ax.scatter(range(0,1713,1), dfRF["rating"],color="red",label="Gerçek Değerler")
ax.plot(range(0,1713,1), dfRF["predict"],color="blue",label="Predict Eğrisi")
ax.set(xlabel='Test Verileri', ylabel='Rating',title='Random Forest')
ax.grid()
plt.legend()
plt.show()

## R2  ##Regresyon algoritmalarımızın karşılaştırılması karşılaştırması
from sklearn.metrics import r2_score

veri = {
  "R2": [r2_score(y_test,RF.predict(X)), r2_score(y_test,DT.predict(X)), r2_score(y_olcekli,svr_reg.predict(x_olcekli)),
        r2_score(y_test,linReg.predict(polyReg.fit_transform(X))), r2_score(y_test,LR.predict(x_test))]
}
pd.DataFrame(data=veri,index = ["Random Forest", "Decision Tree", "SVR","Polynomial Reg","Linear Reg"])